In [3]:
import pandas as pd
import numpy as np
import csv
import re
import seaborn as sns
import gensim
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
import matplotlib.pyplot as plt
from scipy import sparse
from gensim.models.doc2vec import TaggedDocument

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno -3] Temporary failure in name resolution>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [8]:
# load the data
dataset = pd.read_csv("Data Skripsi Anne NoTgl.csv", encoding="latin-1")
dataset = dataset.head()

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Judul       5 non-null      object
 1   Isi Berita  5 non-null      object
 2   Label       5 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 248.0+ bytes


In [7]:
# Create duplicate of data
dataset_copy = dataset.copy()                          
dataset_copy.dropna(inplace = True)
dataset_copy.head()

,Judul,Isi Berita,Label
0,Kapolri Minta Vaksinasi Booster untuk Lansia D...,CIANJUR - Kapolri Jenderal Listyo Sigit Prabow...,0
1,Sudah Vaksin dan Penyintas Covid-19 Masih Terp...,JAKARTA- Gejala pada infeksi Covid-19 variant ...,1
2,Infeksi Covid-19 Tak Hanya Berbahaya bagi Komo...,DOKTER Spesialis Penyakit Dalam Konsultan Peny...,0
3,MNC Peduli dan Lotte Mart Gelar Sentra Vaksin ...,JAKARTA - Lotte Mart bekerja sama dengan MNC P...,0
4,Kapolda Metro Tinjau Vaksinasi Lansia di Alun-...,"BEKASI - Kapolda Metro Jaya, Irjen Fadil Imran...",0


In [9]:
def cleaning(raw_news):
    import nltk
    import re
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    
    raw_news = ' '.join(re.split(r'(?<=[.:;])\s', raw_news)[:4])

    candidates = []
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", raw_news)
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    # 4. Convert the stopwords list to "set" data type.
    stops = set(nltk.corpus.stopwords.words("indonesian"))
    
    # 5. Remove stop words. 
    words = [w for w in  news_words  if not w in stops]
    
    # 6. Lemmentize 
    wordnet_lem = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    
    # 7. Stemming
    stems = [stemmer.stem(w) for w in wordnet_lem ]
    
    # 8. Join the stemmed words back into one string separated by space, and return the result.
    return " ".join(stems)


## Cleaning on Fake News Dataset

In [10]:
dataset_copy['cleaned_title'] = dataset_copy["Judul"].map(lambda x: cleaning(x))
dataset_copy['cleaned_content'] = dataset_copy["Isi Berita"].map(lambda x: cleaning(x))
dataset_copy.head()

,Judul,Isi Berita,Label,cleaned_title,cleaned_content
0,Kapolri Minta Vaksinasi Booster untuk Lansia D...,CIANJUR - Kapolri Jenderal Listyo Sigit Prabow...,0,kapolri vaksinasi booster lansia maksimal,cianjur kapolri jenderal listyo sigit prabowo ...
1,Sudah Vaksin dan Penyintas Covid-19 Masih Terp...,JAKARTA- Gejala pada infeksi Covid-19 variant ...,1,vaksin sintas covid papar omicron,jakarta gejala infeksi covid variant omicron g...
2,Infeksi Covid-19 Tak Hanya Berbahaya bagi Komo...,DOKTER Spesialis Penyakit Dalam Konsultan Peny...,0,infeksi covid bahaya komorbid,dokter spesialis sakit konsultan sakit tropik ...
3,MNC Peduli dan Lotte Mart Gelar Sentra Vaksin ...,JAKARTA - Lotte Mart bekerja sama dengan MNC P...,0,mnc peduli lotte mart gelar sentra vaksin boos...,jakarta lotte mart mnc peduli helat sentra vak...
4,Kapolda Metro Tinjau Vaksinasi Lansia di Alun-...,"BEKASI - Kapolda Metro Jaya, Irjen Fadil Imran...",0,kapolda metro tinjau vaksinasi lansia alun alu...,bekas kapolda metro jaya irjen fadil imran dam...


## Count of sentences in Title and Text

In [14]:
from nltk.tokenize import sent_tokenize

def count_sentences(data):
  data['count_title_sentences'] = data['Judul'].apply(lambda x: len(sent_tokenize(x)))
  data['count_text_sentences'] = data['Isi Berita'].apply(lambda x: len(sent_tokenize(x)))

In [15]:
count_sentences(dataset_copy)
dataset_copy.head()

,Judul,Isi Berita,Label,cleaned_title,cleaned_content,count_title_sentences,count_text_sentences
0,Kapolri Minta Vaksinasi Booster untuk Lansia D...,CIANJUR - Kapolri Jenderal Listyo Sigit Prabow...,0,kapolri vaksinasi booster lansia maksimal,cianjur kapolri jenderal listyo sigit prabowo ...,1,26
1,Sudah Vaksin dan Penyintas Covid-19 Masih Terp...,JAKARTA- Gejala pada infeksi Covid-19 variant ...,1,vaksin sintas covid papar omicron,jakarta gejala infeksi covid variant omicron g...,1,19
2,Infeksi Covid-19 Tak Hanya Berbahaya bagi Komo...,DOKTER Spesialis Penyakit Dalam Konsultan Peny...,0,infeksi covid bahaya komorbid,dokter spesialis sakit konsultan sakit tropik ...,1,18
3,MNC Peduli dan Lotte Mart Gelar Sentra Vaksin ...,JAKARTA - Lotte Mart bekerja sama dengan MNC P...,0,mnc peduli lotte mart gelar sentra vaksin boos...,jakarta lotte mart mnc peduli helat sentra vak...,1,13
4,Kapolda Metro Tinjau Vaksinasi Lansia di Alun-...,"BEKASI - Kapolda Metro Jaya, Irjen Fadil Imran...",0,kapolda metro tinjau vaksinasi lansia alun alu...,bekas kapolda metro jaya irjen fadil imran dam...,1,18


## Generating N-Grams and Their Lengths

In [16]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def ngram(text, n):
    n_grams = ngrams(word_tokenize(text), n)
    return [ '_'.join(grams) for grams in n_grams]

In [19]:
def generate_ngram_features(data):
  data["title_unigram"] = data["cleaned_title"].map(lambda x: ngram(x, 1))
  data["text_unigram"] = data["cleaned_content"].map(lambda x: ngram(x, 1))
  data["count_title_unigram"] = list(data.apply(lambda x: len(x['title_unigram']), axis=1))
  data["count_text_unigram"] = list(data.apply(lambda x: len(x['text_unigram']), axis=1))
  data["unique_count_title_unigram"] = list(data.apply(lambda x: len(set(x['title_unigram'])), axis=1))
  data["unique_count_text_unigram"] = list(data.apply(lambda x: len(set(x['text_unigram'])), axis=1))

  data["title_bigram"] = data["cleaned_title"].map(lambda x: ngram(x, 2))
  data["text_bigram"] = data["cleaned_content"].map(lambda x: ngram(x, 2))
  data["count_title_bigram"] = list(data.apply(lambda x: len(x['title_bigram']), axis=1))
  data["count_text_bigram"] = list(data.apply(lambda x: len(x['text_bigram']), axis=1))
  data["unique_count_title_bigram"] = list(data.apply(lambda x: len(set(x['title_bigram'])), axis=1))
  data["unique_count_text_bigram"] = list(data.apply(lambda x: len(set(x['text_bigram'])), axis=1))

  data["title_trigram"] = data["cleaned_title"].map(lambda x: ngram(x, 3))
  data["text_trigram"] = data["cleaned_content"].map(lambda x: ngram(x, 3))
  data["count_title_trigram"] = list(data.apply(lambda x: len(x['title_trigram']), axis=1))
  data["count_text_trigram"] = list(data.apply(lambda x: len(x['text_trigram']), axis=1))
  data["unique_count_title_trigram"] = list(data.apply(lambda x: len(set(x['title_trigram'])), axis=1))
  data["unique_count_text_trigram"] = list(data.apply(lambda x: len(set(x['text_trigram'])), axis=1))

In [20]:
generate_ngram_features(dataset_copy)
dataset_copy.head()

,Judul,Isi Berita,Label,cleaned_title,cleaned_content,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,...,count_title_bigram,count_text_bigram,unique_count_title_bigram,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram
0,Kapolri Minta Vaksinasi Booster untuk Lansia D...,CIANJUR - Kapolri Jenderal Listyo Sigit Prabow...,0,kapolri vaksinasi booster lansia maksimal,cianjur kapolri jenderal listyo sigit prabowo ...,1,26,"[kapolri, vaksinasi, booster, lansia, maksimal]","[cianjur, kapolri, jenderal, listyo, sigit, pr...",5,...,4,56,4,56,"[kapolri_vaksinasi_booster, vaksinasi_booster_...","[cianjur_kapolri_jenderal, kapolri_jenderal_li...",3,55,3,55
1,Sudah Vaksin dan Penyintas Covid-19 Masih Terp...,JAKARTA- Gejala pada infeksi Covid-19 variant ...,1,vaksin sintas covid papar omicron,jakarta gejala infeksi covid variant omicron g...,1,19,"[vaksin, sintas, covid, papar, omicron]","[jakarta, gejala, infeksi, covid, variant, omi...",5,...,4,30,4,30,"[vaksin_sintas_covid, sintas_covid_papar, covi...","[jakarta_gejala_infeksi, gejala_infeksi_covid,...",3,29,3,29
2,Infeksi Covid-19 Tak Hanya Berbahaya bagi Komo...,DOKTER Spesialis Penyakit Dalam Konsultan Peny...,0,infeksi covid bahaya komorbid,dokter spesialis sakit konsultan sakit tropik ...,1,18,"[infeksi, covid, bahaya, komorbid]","[dokter, spesialis, sakit, konsultan, sakit, t...",4,...,3,45,3,45,"[infeksi_covid_bahaya, covid_bahaya_komorbid]","[dokter_spesialis_sakit, spesialis_sakit_konsu...",2,44,2,44
3,MNC Peduli dan Lotte Mart Gelar Sentra Vaksin ...,JAKARTA - Lotte Mart bekerja sama dengan MNC P...,0,mnc peduli lotte mart gelar sentra vaksin boos...,jakarta lotte mart mnc peduli helat sentra vak...,1,13,"[mnc, peduli, lotte, mart, gelar, sentra, vaks...","[jakarta, lotte, mart, mnc, peduli, helat, sen...",12,...,11,42,11,38,"[mnc_peduli_lotte, peduli_lotte_mart, lotte_ma...","[jakarta_lotte_mart, lotte_mart_mnc, mart_mnc_...",10,41,10,39
4,Kapolda Metro Tinjau Vaksinasi Lansia di Alun-...,"BEKASI - Kapolda Metro Jaya, Irjen Fadil Imran...",0,kapolda metro tinjau vaksinasi lansia alun alu...,bekas kapolda metro jaya irjen fadil imran dam...,1,18,"[kapolda, metro, tinjau, vaksinasi, lansia, al...","[bekas, kapolda, metro, jaya, irjen, fadil, im...",8,...,7,50,7,47,"[kapolda_metro_tinjau, metro_tinjau_vaksinasi,...","[bekas_kapolda_metro, kapolda_metro_jaya, metr...",6,49,6,49


## Common N-grams between Title and Text

In [22]:
def common_ngrams_in_text(data):
  data["count_title_unigrams_in_text"] =  list(data.apply(lambda x: sum([1. for w in x['title_unigram'] if w in set(x['text_unigram'])]), axis=1))
  data["count_title_bigrams_in_text"] =  list(data.apply(lambda x: sum([1. for w in x['title_bigram'] if w in set(x['text_bigram'])]), axis=1))
  data["count_title_trigrams_in_text"] =  list(data.apply(lambda x: sum([1. for w in x['title_trigram'] if w in set(x['text_trigram'])]), axis=1))

In [23]:
common_ngrams_in_text(dataset_copy)
dataset_copy.head()

,Judul,Isi Berita,Label,cleaned_title,cleaned_content,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,...,unique_count_text_bigram,title_trigram,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text
0,Kapolri Minta Vaksinasi Booster untuk Lansia D...,CIANJUR - Kapolri Jenderal Listyo Sigit Prabow...,0,kapolri vaksinasi booster lansia maksimal,cianjur kapolri jenderal listyo sigit prabowo ...,1,26,"[kapolri, vaksinasi, booster, lansia, maksimal]","[cianjur, kapolri, jenderal, listyo, sigit, pr...",5,...,56,"[kapolri_vaksinasi_booster, vaksinasi_booster_...","[cianjur_kapolri_jenderal, kapolri_jenderal_li...",3,55,3,55,5.0,1.0,0.0
1,Sudah Vaksin dan Penyintas Covid-19 Masih Terp...,JAKARTA- Gejala pada infeksi Covid-19 variant ...,1,vaksin sintas covid papar omicron,jakarta gejala infeksi covid variant omicron g...,1,19,"[vaksin, sintas, covid, papar, omicron]","[jakarta, gejala, infeksi, covid, variant, omi...",5,...,30,"[vaksin_sintas_covid, sintas_covid_papar, covi...","[jakarta_gejala_infeksi, gejala_infeksi_covid,...",3,29,3,29,3.0,0.0,0.0
2,Infeksi Covid-19 Tak Hanya Berbahaya bagi Komo...,DOKTER Spesialis Penyakit Dalam Konsultan Peny...,0,infeksi covid bahaya komorbid,dokter spesialis sakit konsultan sakit tropik ...,1,18,"[infeksi, covid, bahaya, komorbid]","[dokter, spesialis, sakit, konsultan, sakit, t...",4,...,45,"[infeksi_covid_bahaya, covid_bahaya_komorbid]","[dokter_spesialis_sakit, spesialis_sakit_konsu...",2,44,2,44,4.0,0.0,0.0
3,MNC Peduli dan Lotte Mart Gelar Sentra Vaksin ...,JAKARTA - Lotte Mart bekerja sama dengan MNC P...,0,mnc peduli lotte mart gelar sentra vaksin boos...,jakarta lotte mart mnc peduli helat sentra vak...,1,13,"[mnc, peduli, lotte, mart, gelar, sentra, vaks...","[jakarta, lotte, mart, mnc, peduli, helat, sen...",12,...,38,"[mnc_peduli_lotte, peduli_lotte_mart, lotte_ma...","[jakarta_lotte_mart, lotte_mart_mnc, mart_mnc_...",10,41,10,39,9.0,4.0,0.0
4,Kapolda Metro Tinjau Vaksinasi Lansia di Alun-...,"BEKASI - Kapolda Metro Jaya, Irjen Fadil Imran...",0,kapolda metro tinjau vaksinasi lansia alun alu...,bekas kapolda metro jaya irjen fadil imran dam...,1,18,"[kapolda, metro, tinjau, vaksinasi, lansia, al...","[bekas, kapolda, metro, jaya, irjen, fadil, im...",8,...,47,"[kapolda_metro_tinjau, metro_tinjau_vaksinasi,...","[bekas_kapolda_metro, kapolda_metro_jaya, metr...",6,49,6,49,8.0,3.0,0.0


## TF_IDF and Cosine Similarities

In [24]:
def concat_title_text(data):
  data['cleaned_title_text'] = data['cleaned_title'] + ' ' + data['cleaned_content']

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf(data):
  concat_title_text(data)
  combined_vectors = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=1, use_idf=True, smooth_idf=True)
  combined_vectors.fit(data["cleaned_title_text"])
  combined_vectors_dictionary = combined_vectors.vocabulary_
  # print (combined_vectors_dictionary)

  # Now lets generate the TfIdf Vectors seperately for Title and Text from the vector dictionary
  # built above
  title_vectors = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=1, use_idf=True, smooth_idf=True, vocabulary=combined_vectors_dictionary)
  title_tfidf_vectors = title_vectors.fit_transform(data['cleaned_title'])
  print (title_tfidf_vectors.shape)
  # print ("\n Title TF-IDF Vectors")
  # print (title_vectors.get_feature_names())     

  text_vectors = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=1, use_idf=True, smooth_idf=True, vocabulary=combined_vectors_dictionary)
  text_tfidf_vectors = text_vectors.fit_transform(data['cleaned_content'])

  # print ("\n Text TF-IDF Vectors")
  # print (text_vectors.get_feature_names())
  print (text_tfidf_vectors.shape)

  return title_tfidf_vectors, text_tfidf_vectors

In [29]:
from scipy.spatial.distance import cosine

def similarity_score(data, title_vectors, text_vectors):
  similarity_score = []
  for i in range(len(data)):
      similarity_score.append(1 - cosine(title_vectors[i], text_vectors[i]))
  return similarity_score

def tf_idf_similarity_score(data):
  title_tfidf_vectors, text_tfidf_vectors = tf_idf(data)
  data['similarity_title_text'] = similarity_score(data, title_tfidf_vectors.toarray(), text_tfidf_vectors.toarray())
  return title_tfidf_vectors, text_tfidf_vectors

In [30]:
stance_title_tfidf_vectors, stance_text_tfidf_vectors = tf_idf_similarity_score(dataset_copy)
dataset_copy.head()

(100, 3335)
(100, 3335)


/home/candra/work_space/tutorial/tests/env3/lib/python3.8/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,Judul,Isi Berita,Label,cleaned_title,cleaned_content,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,...,text_trigram,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text
0,Kapolri Minta Vaksinasi Booster untuk Lansia D...,CIANJUR - Kapolri Jenderal Listyo Sigit Prabow...,0,kapolri vaksinasi booster lansia maksimal,cianjur kapolri jenderal listyo sigit prabowo ...,1,26,"[kapolri, vaksinasi, booster, lansia, maksimal]","[cianjur, kapolri, jenderal, listyo, sigit, pr...",5,...,"[cianjur_kapolri_jenderal, kapolri_jenderal_li...",3,55,3,55,5.0,1.0,0.0,kapolri vaksinasi booster lansia maksimal cian...,0.152944
1,Sudah Vaksin dan Penyintas Covid-19 Masih Terp...,JAKARTA- Gejala pada infeksi Covid-19 variant ...,1,vaksin sintas covid papar omicron,jakarta gejala infeksi covid variant omicron g...,1,19,"[vaksin, sintas, covid, papar, omicron]","[jakarta, gejala, infeksi, covid, variant, omi...",5,...,"[jakarta_gejala_infeksi, gejala_infeksi_covid,...",3,29,3,29,3.0,0.0,0.0,vaksin sintas covid papar omicron jakarta geja...,0.000000
2,Infeksi Covid-19 Tak Hanya Berbahaya bagi Komo...,DOKTER Spesialis Penyakit Dalam Konsultan Peny...,0,infeksi covid bahaya komorbid,dokter spesialis sakit konsultan sakit tropik ...,1,18,"[infeksi, covid, bahaya, komorbid]","[dokter, spesialis, sakit, konsultan, sakit, t...",4,...,"[dokter_spesialis_sakit, spesialis_sakit_konsu...",2,44,2,44,4.0,0.0,0.0,infeksi covid bahaya komorbid dokter spesialis...,0.000000
3,MNC Peduli dan Lotte Mart Gelar Sentra Vaksin ...,JAKARTA - Lotte Mart bekerja sama dengan MNC P...,0,mnc peduli lotte mart gelar sentra vaksin boos...,jakarta lotte mart mnc peduli helat sentra vak...,1,13,"[mnc, peduli, lotte, mart, gelar, sentra, vaks...","[jakarta, lotte, mart, mnc, peduli, helat, sen...",12,...,"[jakarta_lotte_mart, lotte_mart_mnc, mart_mnc_...",10,41,10,39,9.0,4.0,0.0,mnc peduli lotte mart gelar sentra vaksin boos...,0.447214
4,Kapolda Metro Tinjau Vaksinasi Lansia di Alun-...,"BEKASI - Kapolda Metro Jaya, Irjen Fadil Imran...",0,kapolda metro tinjau vaksinasi lansia alun alu...,bekas kapolda metro jaya irjen fadil imran dam...,1,18,"[kapolda, metro, tinjau, vaksinasi, lansia, al...","[bekas, kapolda, metro, jaya, irjen, fadil, im...",8,...,"[bekas_kapolda_metro, kapolda_metro_jaya, metr...",6,49,6,49,8.0,3.0,0.0,kapolda metro tinjau vaksinasi lansia alun alu...,0.307794


## Setup Word2Vec using news corpus and Similarity scores

In [31]:
def cleaning_simple_tokenize(raw_news):
    import nltk
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", raw_news)
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    return np.array(news_words)

In [ ]:
from gensim.models import Word2Vec

Bigger_list=[]
with open('vocab.txt','r') as f:
    for line in f:
        strip_lines=line.strip()
        Bigger_list.append(strip_lines.split())

model= Word2Vec(Bigger_list)
model.save("word2vec.model")
model.save("model.bin")

load_model = Word2Vec.load('model.bin')
words_model = list(google_model.wv.key_to_index.keys())

In [72]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler()

In [73]:
from sklearn.preprocessing import normalize
from functools import reduce

def word2vec_similarity_score(data):
  data['word2vec_cleaned_title'] = data['Judul'].map(lambda x: cleaning_simple_tokenize(x))
  data['word2vec_cleaned_content'] = data['Isi Berita'].map(lambda x: cleaning_simple_tokenize(x))

  title_word2vec = data['word2vec_cleaned_title'].map(lambda words: reduce(np.add, [load_model.wv.get_index(word) for word in words if word in words_model], [0.]*300))
  text_word2vec = data['word2vec_cleaned_content'].map(lambda words: reduce(np.add, [load_model.wv.get_index(word) for word in words if word in words_model], [0.]*300))

  data['word2vec_similarity_title_text'] = similarity_score(data, np.array(title_word2vec), np.array(text_word2vec))

In [75]:
word2vec_similarity_score(dataset_copy)
dataset_copy.head(20)

,Judul,Isi Berita,Label,cleaned_title,cleaned_content,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,...,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text,word2vec_cleaned_title,word2vec_cleaned_content,word2vec_similarity_title_text
0,Kapolri Minta Vaksinasi Booster untuk Lansia D...,CIANJUR - Kapolri Jenderal Listyo Sigit Prabow...,0,kapolri vaksinasi booster lansia maksimal,cianjur kapolri jenderal listyo sigit prabowo ...,1,26,"[kapolri, vaksinasi, booster, lansia, maksimal]","[cianjur, kapolri, jenderal, listyo, sigit, pr...",5,...,3,55,5.0,1.0,0.0,kapolri vaksinasi booster lansia maksimal cian...,0.152944,"[kapolri, minta, vaksinasi, booster, untuk, la...","[cianjur, kapolri, jenderal, listyo, sigit, pr...",1.0
1,Sudah Vaksin dan Penyintas Covid-19 Masih Terp...,JAKARTA- Gejala pada infeksi Covid-19 variant ...,1,vaksin sintas covid papar omicron,jakarta gejala infeksi covid variant omicron g...,1,19,"[vaksin, sintas, covid, papar, omicron]","[jakarta, gejala, infeksi, covid, variant, omi...",5,...,3,29,3.0,0.0,0.0,vaksin sintas covid papar omicron jakarta geja...,0.000000,"[sudah, vaksin, dan, penyintas, covid, masih, ...","[jakarta, gejala, pada, infeksi, covid, varian...",1.0
2,Infeksi Covid-19 Tak Hanya Berbahaya bagi Komo...,DOKTER Spesialis Penyakit Dalam Konsultan Peny...,0,infeksi covid bahaya komorbid,dokter spesialis sakit konsultan sakit tropik ...,1,18,"[infeksi, covid, bahaya, komorbid]","[dokter, spesialis, sakit, konsultan, sakit, t...",4,...,2,44,4.0,0.0,0.0,infeksi covid bahaya komorbid dokter spesialis...,0.000000,"[infeksi, covid, tak, hanya, berbahaya, bagi, ...","[dokter, spesialis, penyakit, dalam, konsultan...",1.0
3,MNC Peduli dan Lotte Mart Gelar Sentra Vaksin ...,JAKARTA - Lotte Mart bekerja sama dengan MNC P...,0,mnc peduli lotte mart gelar sentra vaksin boos...,jakarta lotte mart mnc peduli helat sentra vak...,1,13,"[mnc, peduli, lotte, mart, gelar, sentra, vaks...","[jakarta, lotte, mart, mnc, peduli, helat, sen...",12,...,10,39,9.0,4.0,0.0,mnc peduli lotte mart gelar sentra vaksin boos...,0.447214,"[mnc, peduli, dan, lotte, mart, gelar, sentra,...","[jakarta, lotte, mart, bekerja, sama, dengan, ...",1.0
4,Kapolda Metro Tinjau Vaksinasi Lansia di Alun-...,"BEKASI - Kapolda Metro Jaya, Irjen Fadil Imran...",0,kapolda metro tinjau vaksinasi lansia alun alu...,bekas kapolda metro jaya irjen fadil imran dam...,1,18,"[kapolda, metro, tinjau, vaksinasi, lansia, al...","[bekas, kapolda, metro, jaya, irjen, fadil, im...",8,...,6,49,8.0,3.0,0.0,kapolda metro tinjau vaksinasi lansia alun alu...,0.307794,"[kapolda, metro, tinjau, vaksinasi, lansia, di...","[bekasi, kapolda, metro, jaya, irjen, fadil, i...",1.0
5,"Cirebon PPKM Level 4, tapi Tingkat Hunian Ruma...",CIREBON - Berdasarkan asesmen Kementerian Kese...,0,cirebon ppkm level tingkat huni rumah sakit ke...,cirebon dasar asesmen menteri sehat kota cireb...,1,15,"[cirebon, ppkm, level, tingkat, huni, rumah, s...","[cirebon, dasar, asesmen, menteri, sehat, kota...",8,...,6,42,4.0,1.0,0.0,cirebon ppkm level tingkat huni rumah sakit ke...,0.000000,"[cirebon, ppkm, level, tapi, tingkat, hunian, ...","[cirebon, berdasarkan, asesmen, kementerian, k...",1.0
6,"Melawan Covid-19, Psikolog: Berpikir Sehat dan...","PSIKOLOG Dr Novrans Eka Saputra mengatakan, be...",0,lawan covid psikolog pikir sehat cemas,psikolog dr novrans eka saputra pikir sehat ce...,1,15,"[lawan, covid, psikolog, pikir, sehat, cemas]","[psikolog, dr, novrans, eka, saputra, pikir, s...",6,...,4,44,6.0,3.0,1.0,lawan covid psikolog pikir sehat cemas psikolo...,0.311805,"[melawan, covid, psikolog, berpikir, sehat, da...","[psikolog, dr, novrans, eka, saputra, mengatak...",1.0
7,"Pandemi Covid-19, BNPB Fokus Pada Perubahan Pe...",JAKARTA - Kepala Badan Nasional Penanggulangan...,0,pandemi covid bnpb fokus ubah perilaku,jak

## LSA Topic Modelling using SVD and Cosine Similarities

In [158]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import vstack

def svd(data, title_tfidf_vectors, text_tfidf_vectors):
  truncated_svd = TruncatedSVD(n_components=50, n_iter=10)
  combined_vectors = vstack([title_tfidf_vectors, text_tfidf_vectors])
  truncated_svd.fit(combined_vectors)

  title_svd = truncated_svd.transform(title_tfidf_vectors)
  print (title_svd.shape)
  text_svd = truncated_svd.transform(text_tfidf_vectors)
  print (text_svd.shape)
  return title_svd, text_svd

def topic_similarity_score(data, title_tfidf_vectors, text_tfidf_vectors):
  title_svd_vectors, text_svd_vectors = svd(data, title_tfidf_vectors, text_tfidf_vectors)
  data['topics_similarity_title_text'] = similarity_score(data, title_svd_vectors, text_svd_vectors)

In [77]:
topic_similarity_score(dataset_copy, stance_title_tfidf_vectors, stance_text_tfidf_vectors)
dataset_copy.head()

(100, 100)
(100, 100)


/home/candra/work_space/tutorial/tests/env3/lib/python3.8/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


,Judul,Isi Berita,Label,cleaned_title,cleaned_content,count_title_sentences,count_text_sentences,title_unigram,text_unigram,count_title_unigram,...,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,cleaned_title_text,similarity_title_text,word2vec_cleaned_title,word2vec_cleaned_content,word2vec_similarity_title_text,topics_similarity_title_text
0,Kapolri Minta Vaksinasi Booster untuk Lansia D...,CIANJUR - Kapolri Jenderal Listyo Sigit Prabow...,0,kapolri vaksinasi booster lansia maksimal,cianjur kapolri jenderal listyo sigit prabowo ...,1,26,"[kapolri, vaksinasi, booster, lansia, maksimal]","[cianjur, kapolri, jenderal, listyo, sigit, pr...",5,...,55,5.0,1.0,0.0,kapolri vaksinasi booster lansia maksimal cian...,0.152944,"[kapolri, minta, vaksinasi, booster, untuk, la...","[cianjur, kapolri, jenderal, listyo, sigit, pr...",1.0,0.966414
1,Sudah Vaksin dan Penyintas Covid-19 Masih Terp...,JAKARTA- Gejala pada infeksi Covid-19 variant ...,1,vaksin sintas covid papar omicron,jakarta gejala infeksi covid variant omicron g...,1,19,"[vaksin, sintas, covid, papar, omicron]","[jakarta, gejala, infeksi, covid, variant, omi...",5,...,29,3.0,0.0,0.0,vaksin sintas covid papar omicron jakarta geja...,0.000000,"[sudah, vaksin, dan, penyintas, covid, masih, ...","[jakarta, gejala, pada, infeksi, covid, varian...",1.0,-0.136183
2,Infeksi Covid-19 Tak Hanya Berbahaya bagi Komo...,DOKTER Spesialis Penyakit Dalam Konsultan Peny...,0,infeksi covid bahaya komorbid,dokter spesialis sakit konsultan sakit tropik ...,1,18,"[infeksi, covid, bahaya, komorbid]","[dokter, spesialis, sakit, konsultan, sakit, t...",4,...,44,4.0,0.0,0.0,infeksi covid bahaya komorbid dokter spesialis...,0.000000,"[infeksi, covid, tak, hanya, berbahaya, bagi, ...","[dokter, spesialis, penyakit, dalam, konsultan...",1.0,0.107184
3,MNC Peduli dan Lotte Mart Gelar Sentra Vaksin ...,JAKARTA - Lotte Mart bekerja sama dengan MNC P...,0,mnc peduli lotte mart gelar sentra vaksin boos...,jakarta lotte mart mnc peduli helat sentra vak...,1,13,"[mnc, peduli, lotte, mart, gelar, sentra, vaks...","[jakarta, lotte, mart, mnc, peduli, helat, sen...",12,...,39,9.0,4.0,0.0,mnc peduli lotte mart gelar sentra vaksin boos...,0.447214,"[mnc, peduli, dan, lotte, mart, gelar, sentra,...","[jakarta, lotte, mart, bekerja, sama, dengan, ...",1.0,0.999995
4,Kapolda Metro Tinjau Vaksinasi Lansia di Alun-...,"BEKASI - Kapolda Metro Jaya, Irjen Fadil Imran...",0,kapolda metro tinjau vaksinasi lansia alun alu...,bekas kapolda metro jaya irjen fadil imran dam...,1,18,"[kapolda, metro, tinjau, vaksinasi, lansia, al...","[bekas, kapolda, metro, jaya, irjen, fadil, im...",8,...,49,8.0,3.0,0.0,kapolda metro tinjau vaksinasi lansia alun alu...,0.307794,"[kapolda, metro, tinjau, vaksinasi, lansia, di...","[bekasi, kapolda, metro, jaya, irjen, fadil, i...",1.0,0.999422


## Features Distillation for Modelling

In [81]:
from sklearn import preprocessing

distilled_features_cols = ['Label', 'count_title_sentences', 'count_text_sentences', 'count_title_unigram', 'count_text_unigram',
                'unique_count_title_unigram', 'unique_count_text_unigram', 'count_title_bigram', 'count_text_bigram',
                'unique_count_title_bigram', 'unique_count_text_bigram', 'count_title_trigram', 'count_text_trigram',
                'unique_count_title_trigram', 'unique_count_text_trigram', 'count_title_unigrams_in_text', 'count_title_bigrams_in_text',
                'count_title_trigrams_in_text', 'similarity_title_text', 'topics_similarity_title_text', 'word2vec_similarity_title_text']

df_final_multi = dataset_copy[distilled_features_cols]
le = preprocessing.LabelEncoder()
le.fit(df_final_multi['Label'])
df_final_multi['Label'] = le.transform(df_final_multi['Label'])
df_final_multi.head()

<ipython-input-81-ace9d08d809a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_multi['Label'] = le.transform(df_final_multi['Label'])


,Label,count_title_sentences,count_text_sentences,count_title_unigram,count_text_unigram,unique_count_title_unigram,unique_count_text_unigram,count_title_bigram,count_text_bigram,unique_count_title_bigram,...,count_title_trigram,count_text_trigram,unique_count_title_trigram,unique_count_text_trigram,count_title_unigrams_in_text,count_title_bigrams_in_text,count_title_trigrams_in_text,similarity_title_text,topics_similarity_title_text,word2vec_similarity_title_text
0,0,1,26,5,57,5,46,4,56,4,...,3,55,3,55,5.0,1.0,0.0,0.152944,0.966414,1.0
1,1,1,19,5,31,5,26,4,30,4,...,3,29,3,29,3.0,0.0,0.0,0.000000,-0.136183,1.0
2,0,1,18,4,46,4,36,3,45,3,...,2,44,2,44,4.0,0.0,0.0,0.000000,0.107184,1.0
3,0,1,13,12,43,12,32,11,42,11,...,10,41,10,39,9.0,4.0,0.0,0.447214,0.999995,1.0
4,0,1,18,8,51,7,36,7,50,7,...,6,49,6,49,8.0,3.0,0.0,0.307794,0.999422,1.0


## Model Training

In [113]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, accuracy_score, \
    classification_report, precision_recall_curve, roc_curve, auc, average_precision_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split, cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [114]:
multi_classifiers = {
    "LogisticRegression": (LogisticRegression(max_iter=10000, solver='lbfgs')),
    "KNearest": (KNeighborsClassifier()),
    "Support Vector Classifier": (SVC()),
    "DecisionTreeClassifier": (DecisionTreeClassifier()),
    "Naive Bayes": (GaussianNB()),
    "Random forests": (RandomForestClassifier()),
    "XGBoost Classifier": (XGBClassifier()),
    "GradientBoostingClassifier": (GradientBoostingClassifier()),
    "MLP Classifier": (MLPClassifier())
}

In [115]:
binary_classifiers = {
    "LogisticRegression": (LogisticRegression(max_iter=10000, solver='lbfgs')),
    "KNearest": (KNeighborsClassifier()),
    "Support Vector Classifier": (SVC()),
    "DecisionTreeClassifier": (DecisionTreeClassifier()),
    "Naive Bayes": (GaussianNB()),
    "Random forests": (RandomForestClassifier()),
    "XGBoost Classifier": (XGBClassifier()),
    "GradientBoostingClassifier": (GradientBoostingClassifier()),
    "MLP Classifier": (MLPClassifier())
}

In [116]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

def run_all_models(folds, X_final_train, y_final_train, X_final_test, y_final_test, classifiers):
    for classifier_name, classifier in classifiers.items():
        k_fold = StratifiedKFold(n_splits=folds, random_state=100, shuffle=True)
        cross_val_scores = []
        precision_scores = []
        recall_scores = []
        roc_auc_scores = []
        f1_scores = []
        for train_index, test_index in k_fold.split(X_final_train, y_final_train):
            X_train, X_test = pd.DataFrame(data=X_final_train, index=train_index), pd.DataFrame(data=X_final_train, index=test_index)
            y_train, y_test = pd.DataFrame(data=y_final_train, index=train_index), pd.DataFrame(data=y_final_train, index=test_index)
            model = classifier

            model.fit(X_train, y_train.values.ravel())
            scores = cross_val_score(model, X_train, y_train.values.ravel(), cv=5)
            cross_val_scores.append(scores)
            y_pred = model.predict(X_test)
            precision_scores.append(precision_score(y_test, y_pred, average=None))
            recall_scores.append(recall_score(y_test, y_pred, average=None))
            # roc_auc_scores.append(roc_auc_score(y_test, model.predict_proba(X_test), average='macro', multi_class='ovo'))
            f1_scores.append(f1_score(y_test, y_pred, average=None))

        print('============================= {} ============================='.format(classifier_name))
        print('Mean cross validation score: {}'.format(np.array([cross_val_scores]).mean()))
        print('Mean precision score: {}'.format(np.array([precision_scores]).mean()))
        print('Mean Recall score: {}'.format(np.array([recall_scores]).mean()))
        # print('Mean ROC-AUC score: {}'.format(np.array([roc_auc_scores]).mean()))
        print('Mean F1 score: {}'.format(np.array([f1_scores]).mean()))
        print('******* Real test dataset metrics *******')
        y_final_pred = model.predict((X_final_test))
        print('Accuracy score for the real test set:\n', accuracy_score(y_final_test, y_final_pred))
        print('confusion matrix for the real test set:\n', confusion_matrix(y_final_test, y_final_pred))
        print('Classification report for the real test set:\n', classification_report(y_final_test, y_final_pred))

## AUC-ROC Curves

In [117]:
def plot_auc_roc_curve(y_test, y_pred, name):
    fpr, tpr, thresholds = roc_curve(y_test.to_numpy(), y_pred)
    roc_auc = auc(fpr, tpr)

    # Plot ROC
    plt.title('ROC for {}'.format(name))
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([-0.1, 1.0])
    plt.ylim([-0.1, 1.01])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

## Precision Recall Curve

In [118]:
def plot_precision_recall_curve(y_test, y_pred_prob, name):
    # Generate precision recall curve values: precision, recall, thresholds
    precision, recall, thresholds = precision_recall_curve(y_test.to_numpy(), y_pred_prob)

    # Plot Precision Recall curve
    plt.plot(recall, precision)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    avg_precision_score = average_precision_score(y_test, y_pred_prob)
    plt.title('PRC for {} - avg precision score: {}'.format(name, str(avg_precision_score)))
    plt.show()

## Multiclass Classification Results

In [ ]:
# df_final_multi.fillna(0, inplace=True)

# y = df_final_multi['Label']
# X = df_final_multi.drop(['Label'], axis=1)

# X.reset_index(drop=True, inplace=True)
# y.reset_index(drop=True, inplace=True)

# X_final_train, X_final_test, y_final_train, y_final_test = train_test_split(X, y, test_size=0.20, random_state=100, stratify=y)
# X_final_train.reset_index(drop=True, inplace=True)
# X_final_test.reset_index(drop=True, inplace=True)
# y_final_train.reset_index(drop=True, inplace=True)
# y_final_test.reset_index(drop=True, inplace=True)

# run_all_models(5, X_final_train, y_final_train, X_final_test, y_final_test, multi_classifiers)

## Binary classification Results

In [140]:
df_final_multi.fillna(0, inplace=True)

y = df_final_multi['Label']
X = df_final_multi.drop(['Label'], axis=1)

X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

X_final_train, X_final_test, y_final_train, y_final_test = train_test_split(X, y, test_size=0.20, random_state=100, stratify=y)
X_final_train.reset_index(drop=True, inplace=True)
X_final_test.reset_index(drop=True, inplace=True)
y_final_train.reset_index(drop=True, inplace=True)
y_final_test.reset_index(drop=True, inplace=True)

run_all_models(5, X_final_train, y_final_train, X_final_test, y_final_test, binary_classifiers)

============================= LogisticRegression =============================
Mean cross validation score: 0.6953846153846154
Mean precision score: 0.47833333333333333
Mean Recall score: 0.48589743589743584
Mean F1 score: 0.45379894179894176
******* Real test dataset metrics *******
Accuracy score for the real test set:
 0.75
confusion matrix for the real test set:
 [[14  1]
 [ 4  1]]
Classification report for the real test set:
               precision    recall  f1-score   support

           0       0.78      0.93      0.85        15
           1       0.50      0.20      0.29         5

    accuracy                           0.75        20
   macro avg       0.64      0.57      0.57        20
weighted avg       0.71      0.75      0.71        20

============================= KNearest =============================
Mean cross validation score: 0.7187179487179488
Mean precision score: 0.48142857142857143
Mean Recall score: 0.5006410256410256
Mean F1 score: 0.462954822954823
******* 

## Prediction and Testing

In [161]:
def get_distilled_dataset(title, text):
  data = {'Judul': [title], 'Isi Berita': [text]}
  df_test = pd.DataFrame(data)

	# Add the cleaned columns
  df_test['cleaned_title'] = df_test["Judul"].map(lambda x: cleaning(x))
  df_test['cleaned_content'] = df_test["Isi Berita"].map(lambda x: cleaning(x))

	# Add sentence count columns
  count_sentences(df_test)

	# Add ngram features
  generate_ngram_features(df_test)

	# Add common ngram counts
  common_ngrams_in_text(df_test)

	# Add TF_IDF and similarity scores
  title_tfidf_vectors, text_tfidf_vectors = tf_idf_similarity_score(df_test)

	# Add word2vec similarity scores
  word2vec_similarity_score(df_test)

	# Add Topic Similarity score
  topic_similarity_score(df_test, title_tfidf_vectors, text_tfidf_vectors)

	# Distill features
  X_cols = [x for i,x in enumerate(distilled_features_cols) if x!='Label']
  return df_test[X_cols]

## Pickle the models

In [163]:
# import pickle
# multi_class_model = multi_classifiers['Random forests']
# multi_class_model_path = 'multi_class_model.pickle'
# pickle.dump(multi_class_model, open(multi_class_model_path, 'wb'))

import pickle

binary_class_model = binary_classifiers['GradientBoostingClassifier']
binary_class_model_path = 'binary_class_model.pickle'
pickle.dump(binary_class_model, open(binary_class_model_path, 'wb'))


In [164]:
# multi_class_model = pickle.load(open(multi_class_model_path, "rb"))
binary_class_model = pickle.load(open(binary_class_model_path, "rb"))

In [165]:
# def predict_multiclass(title, text):
#   df_test = get_distilled_dataset(title, text)
#   return multi_classifiers['Random forests'].predict(df_test), multi_classifiers['Random forests'].predict_proba(df_test)


def predict_binaryclass(title, text):
  df_test = get_distilled_dataset(title, text)
  return binary_classifiers['GradientBoostingClassifier'].predict(df_test), binary_classifiers['GradientBoostingClassifier'].predict_proba(df_test)


In [169]:
def Shecodes_getTitleVsBodyScore(title, text):
#   prediction_multi, prediction_multi_prob = predict_multiclass(title, text)
#   print (prediction_multi)
#   print ("Multi label predicted classification: {}".format(le.inverse_transform(prediction_multi)[0]))
#   print ("Multi label prediction probabilities {} : {}".format(le.inverse_transform([0,1]), prediction_multi_prob))

  prediction_binary, prediction_binary_prob = predict_binaryclass(title, text)
  print (prediction_binary)
  print ("Binary predicted classification: {}".format("Fake" if prediction_binary[0] == 1 else "True"))
  print ("Binary label prediction probabilities ['True', 'Fake']: {}".format(prediction_binary_prob))
  print (prediction_binary)

## Predict

In [172]:
score = Shecodes_getTitleVsBodyScore (
  "Kapolri Minta Vaksinasi Booster untuk Lansia Dimaksimalkan",
   "CIANJUR - Kapolri Jenderal Listyo Sigit Prabowo meminta kepada jajaran TNI, Polri, Forkopimda dan seluruh pihak terkait untuk memaksimalkan akselerasi percepatan vaksinasi booster atau dosisi ketiga bagi kelompok lanjut usia (lansia). Hal itu disampaikan Sigit saat meninjau pelaksanaan akselerasi vaksinasi di seluruh Provinsi Indonesia, dengan menghadiri secara langsung di Obyek Wisata Taman Bunga Nusantara, Kabupaten Cianjur, Jawa Barat, Rabu (23/2/2022). ""Dalam kesempatan ini juga, kembali saya ingatkan terhadap masyarakat khususnya lansia sesuai Surat Edaran dari Kemenkes bahwa sudah diberikan kesempatan bagi yang sudah tiga bulan untuk bisa melaksanakan vaksin booster ketiga. Tentunya harapan saya ini betul-betul bisa dimaksimalkan,"" kata Sigit dalam kunjungannya. Kementerian Kesehatan (Kemenkes) sudah menyatakan waktu pelaksanaan vaksinasi booster sudah bisa dilakukan dalam waktu tiga bulan setelah mendapatkan vaksinasi kedua bagi kelompok lanjut usia. Dalam satu minggu kedepan, Sigit mengungkapkan, akan mengejar target dari vaksinasi booster bagi kelompok lansia. Menurutnya, dengan mendapatkan dosis ketiga itu, masyarakat khususnya yang sudah berusia dan memiliki komorbid, akan memiliki imunitas lebih kuat dari paparan virus Covid-19. ""Karena kalau kita lihat bahwa data-data yang rentan memiliki angka fatalitas tinggi tentunya adalah usia lansia yang disertai komorbid ataupun yang vaksinnya belum lengkap. Jadi ini saya harapkan untuk terus bisa dikerjakan,"" ujar Sigit. Mantan Kabareskrim Polri itu menekankan, untuk mencegah laju pertumbuhan Covid-19 saat ini, khususnya varian Omicron, vaksinasi menjadi upaya penting dalam penanganan dan pengendalian Pandemi Covid-19. Sebab itu, kata Sigit, kedepannya, capaian target akselerasi vaksinasi dalam rangka mewujudkan kekebalan komunal, akan ditingkatkan jumlah, dari 1,1 juta menjadi 1,6 juta vaksinasi per hari. ""Secara nasional terjadi peningkatan vaksinasi kemarin kita berada di angka 1,1 juta. Hari ini target kita 1,6 juta. Ini akan terus kita jaga dan kita pertahankan, serta kita tingkatkan,"" ucap eks Kapolda Banten tersebut. Dengan begitu, Sigit berharap, bisa menghadapi perkembangan pertumbuhan Covid-19 jenis Omicron. Mengingat, menurut Sigit, dari data yang ada tingkat kesembuhan saat ini secara rata-rata nasonal berada di angka 80 hingga 90 persen. ""Tingkat kematian bisa kita jaga walaupun di beberapa wilayah ada yang diangka 5 atau 6 persen. Tapi rata-rata masih berada di angka tiga atau bahkan ada juga yang di bawah angka tiga persen. Ini adalah modal kita untuk kemudian melewati situasi Pandemi khususnya varian baru Omicron,"" tutur Sigit. Sigit optimis tren-tren positif itu bisa dipertahankan dan ditingkatkan. Pasalnya, lanjut Sigit, Indonesia pernah menjadi salah satu negara terbaik dalam rangka penanganan dan pengendalian Pandemi Covid-19. ""Ini harus kita jaga terus, karena ini berdampak tentunya bagi aktivitas masyarakat dan tentunya akan membawa pertumbuhan positif terhadap pertumbuhan ekonomi. Harapan kita bisa menjaga target pemerintah  diangka 5,5 persen. Ini bisa kita jaga,"" papar Sigit. Tak hanya meninjau, Sigit juga memberikan pengarahan tentang upaya dan strategi menekan laju pertumbuhan Covid-19 di seluruh Indonesia, dengan sambungan virtual. #kapolri #Vaksinasi Lansia #Covid-19") 
print(score)

(1, 106)
(1, 106)
(1, 2)
(1, 2)
[0]
Binary predicted classification: True
Binary label prediction probabilities ['True', 'Fake']: [[0.99001249 0.00998751]]
[0]
None
